In [ ]:
# 22
import os
import re
import shutil

# ──────────────────────────────────────────────────────
# 1) 경로 설정
# ──────────────────────────────────────────────────────
SRC_LABEL_DIR = "tree/Datasets/tree 200/labels/train"
DST_ROOT_DIR  = "tree/Datasets/tree200_converted"  
# 별도 폴더에 images/labels/train/val 구조로 보관

# ──────────────────────────────────────────────────────
# 2) 대상 이미지 폴더(기존) 및 확장자
# ──────────────────────────────────────────────────────
MY_IMG_DIR    = "tree/Datasets/250630_split/images"
IMG_EXTS      = (".jpg", ".png")

# ──────────────────────────────────────────────────────
# 3) ID 추출용 정규식: 숫자 5자리 등
# ──────────────────────────────────────────────────────
label_pattern = re.compile(r"(\d{5})")

# ──────────────────────────────────────────────────────
# 4) 새 이미지명 맵: ID → "나무_8_남_XXXXX.jpg"
# ──────────────────────────────────────────────────────
img_id_map = {}
for split in ("train","val"):
    img_dir = os.path.join(MY_IMG_DIR, split)
    for fn in os.listdir(img_dir):
        if fn.lower().endswith(IMG_EXTS):
            m = label_pattern.search(fn)
            if m:
                img_id_map[m.group(1)] = fn

# ──────────────────────────────────────────────────────
# 5) 출력 폴더 구조 생성
# ──────────────────────────────────────────────────────
for sub in ("images/train","images/val","labels/train","labels/val"):
    os.makedirs(os.path.join(DST_ROOT_DIR, sub), exist_ok=True)

# ──────────────────────────────────────────────────────
# 6) 라벨 변환 & 복사
# ──────────────────────────────────────────────────────
for lbl_fn in os.listdir(SRC_LABEL_DIR):
    if not lbl_fn.endswith(".txt"):
        continue
    m = label_pattern.search(lbl_fn)
    if not m:
        continue
    img_id = m.group(1)
    if img_id not in img_id_map:
        # 매핑 이미지 없으면 스킵
        continue

    # 원본 이미지명 기준 새 이름
    new_img_fn = img_id_map[img_id]
    base, _    = os.path.splitext(new_img_fn)
    new_lbl_fn = f"{base}.txt"

    # split 결정: train/val 중 어느 폴더에 있는지 찾아서
    split = "train" if os.path.exists(os.path.join(MY_IMG_DIR,"train",new_img_fn)) else "val"

    # src 라벨, dst 라벨 경로
    src_lbl = os.path.join(SRC_LABEL_DIR, lbl_fn)
    dst_lbl = os.path.join(DST_ROOT_DIR, "labels", split, new_lbl_fn)
    shutil.copy2(src_lbl, dst_lbl)

    # 이미지도 같은 구조로 복사 (원본 훼손 없음)
    src_img = os.path.join(MY_IMG_DIR, split, new_img_fn)
    dst_img = os.path.join(DST_ROOT_DIR, "images", split, new_img_fn)
    shutil.copy2(src_img, dst_img)

    print(f"✓ [{split}] {lbl_fn} → {new_lbl_fn}")

print("✅ 완료: 별도 폴더에 변환된 이미지·라벨링 생성됨.")


✅ 완료: 별도 폴더에 변환된 이미지·라벨링 생성됨.


In [2]:
import os, re

SRC_LABEL_DIR = "tree/Datasets/tree 200/labels/train"
MY_IMG_DIR    = "tree/Datasets/250630_split/images"
label_pattern = re.compile(r"(\d{5})")

# 1) 라벨 ID 목록
label_ids = []
for fn in os.listdir(SRC_LABEL_DIR):
    m = label_pattern.search(fn)
    if m:
        label_ids.append(m.group(1))
print("▶ SRC_LABEL_DIR에서 추출된 ID들:", sorted(label_ids))

# 2) 내 이미지에서 지원되는 ID 목록
img_ids = []
for split in ("train","val"):
    path = os.path.join(MY_IMG_DIR, split)
    if not os.path.isdir(path):
        print(f"⚠ 경로 없음: {path}")
        continue
    for fn in os.listdir(path):
        m = label_pattern.search(fn)
        if m:
            img_ids.append(m.group(1))
print("▶ MY_IMG_DIR에서 추출된 ID들:", sorted(set(img_ids)))

# 3) 차집합 확인
missing = set(label_ids) - set(img_ids)
if missing:
    print("❌ 매핑 대상 이미지를 찾지 못한 ID들:", sorted(missing))
else:
    print("✅ 모든 라벨 ID에 대응하는 이미지가 존재합니다.")


▶ SRC_LABEL_DIR에서 추출된 ID들: ['08017', '08032', '08034', '08040', '08083', '08196', '08239', '08242', '08254', '08261', '08263', '08318', '08345', '08346', '08361', '08366', '08393', '08418', '08450', '08454', '08519', '08532', '08561', '08631', '08668', '08683', '08712', '08745', '08748', '08754', '08773', '08784', '08800', '08805', '08811', '08822', '08845', '08848', '08880', '08889', '08906', '08928', '08955', '08991', '09043', '09115', '09122', '09165', '09175', '09225', '09280', '09335', '09368', '09380', '09402', '09454', '09472', '09483', '09561', '09569', '09578', '09605', '09613', '09618', '09622', '09707', '09746', '09776', '09779', '09797', '09838', '09844', '09880', '09937', '09967', '09988', '10024', '10031', '10032', '10051', '10064', '10082', '10091', '10092', '10106', '10112', '10161', '10253', '10272', '10360', '10381', '10388', '10407', '10448', '10457', '10480', '10486', '10506', '10511', '10549', '10550', '10565', '10566', '10608', '10645', '10688', '10691', '10735', 

# 추가한 데이터셋을 기존 데이터셋에 추가
# 경로를 다시 지정해줄 것.

In [ ]:
import os
import shutil
import random

# ──────────────────────────────────────
# 0) 환경에 맞게 경로만 수정
# ──────────────────────────────────────
SRC_IMG_DIR  = "tree/Prediction/tree_0_detected"    # 추가할 원본 이미지 폴더
# 수동 라벨링된 .txt 파일들이 모두 들어있는 폴더
# (train/val 구분 없이 한 곳에 몰려있음)
SRC_LBL_DIR  = "tree/Datasets/0_tree_detected_5/labels/train"

DST_IMG_ROOT = "tree/Datasets/250630_split/images" # 기존 train/val 상위
DST_LBL_ROOT = "tree/Datasets/250630_split/labels"
TRAIN_RATIO  = 0.8

# ──────────────────────────────────────
# 1) 원본 파일 목록 준비
# ──────────────────────────────────────
all_images = [
    f for f in os.listdir(SRC_IMG_DIR)
    if f.lower().endswith((".jpg", ".png"))
]
random.seed(42)
random.shuffle(all_images)

split_idx  = int(len(all_images) * TRAIN_RATIO)
train_imgs = all_images[:split_idx]
val_imgs   = all_images[split_idx:]

print(f"추가할 이미지 총 {len(all_images)}장 → train: {len(train_imgs)}장, val: {len(val_imgs)}장")

# ──────────────────────────────────────
# 2) 안전 복사 함수
# ──────────────────────────────────────
def safe_copy(src, dst):
    if not os.path.isfile(src):
        print(f"⚠️ 원본 누락: {src}")
        return False
    os.makedirs(os.path.dirname(dst), exist_ok=True)
    if os.path.exists(dst):
        print(f"⚠️ 이미 존재: {dst} (스킵)")
        return False
    shutil.copy2(src, dst)
    return True

# ──────────────────────────────────────
# 3) Dry-run vs 실제 실행 설정
# ──────────────────────────────────────
dry_run = True  # True면 실제 복사 없이 로그만 출력. 완료 후 False로 변경.

# ──────────────────────────────────────
# 4) train/val 폴더에 복사
# ──────────────────────────────────────
for split, img_list in [("train", train_imgs), ("val", val_imgs)]:
    dst_img_dir = os.path.join(DST_IMG_ROOT, split)
    dst_lbl_dir = os.path.join(DST_LBL_ROOT, split)

    for img_name in img_list:
        # 원본 이미지 경로
        src_img = os.path.join(SRC_IMG_DIR, img_name)
        dst_img = os.path.join(dst_img_dir, img_name)

        # 원본 라벨 경로 (단일 SRC_LBL_DIR에서 가져옴)
        lbl_name = os.path.splitext(img_name)[0] + ".txt"
        src_lbl   = os.path.join(SRC_LBL_DIR, lbl_name)
        dst_lbl   = os.path.join(dst_lbl_dir, lbl_name)

        if dry_run:
            print(f"[DRY-RUN] 이미지 → {src_img} -> {dst_img}")
            print(f"[DRY-RUN] 레이블 → {src_lbl} -> {dst_lbl}")
        else:
            ok_img = safe_copy(src_img, dst_img)
            ok_lbl = safe_copy(src_lbl, dst_lbl)
            if ok_img and ok_lbl:
                print(f"✅ {split}에 추가된 파일: {img_name}")

if dry_run:
    print("\n✅ Dry-run 완료. dry_run=False 로 설정 후 다시 실행하면 실제로 복사됩니다.")
else:
    print(f"\n✅ 경로에 복사되었습니다.")

추가할 이미지 총 15장 → train: 12장, val: 3장
⚠️ 이미 존재: tree/Datasets/250630_split/images\train\나무_13_여_05878.jpg (스킵)
⚠️ 이미 존재: tree/Datasets/250630_split/images\train\나무_9_남_06017.jpg (스킵)
⚠️ 이미 존재: tree/Datasets/250630_split/images\train\나무_13_여_05136.jpg (스킵)
⚠️ 이미 존재: tree/Datasets/250630_split/images\train\나무_13_여_04381.jpg (스킵)
⚠️ 이미 존재: tree/Datasets/250630_split/images\train\나무_9_여_02102.jpg (스킵)
⚠️ 이미 존재: tree/Datasets/250630_split/images\train\나무_9_남_04727.jpg (스킵)
⚠️ 이미 존재: tree/Datasets/250630_split/images\train\나무_13_남_05123.jpg (스킵)
⚠️ 이미 존재: tree/Datasets/250630_split/images\train\나무_11_남_07882.jpg (스킵)
⚠️ 이미 존재: tree/Datasets/250630_split/images\train\나무_13_여_12295.jpg (스킵)
⚠️ 이미 존재: tree/Datasets/250630_split/images\train\나무_12_남_01749.jpg (스킵)
⚠️ 이미 존재: tree/Datasets/250630_split/images\train\나무_12_여_07927.jpg (스킵)
⚠️ 이미 존재: tree/Datasets/250630_split/images\train\나무_8_남_09483.jpg (스킵)
⚠️ 이미 존재: tree/Datasets/250630_split/images\val\나무_10_남_08275.jpg (스킵)
⚠️ 이미 존재: tree/Datase